# String Similarity Automation

## Importing Libraries

In [9]:
import pandas as pd
import numpy as np 
from scipy.special import softmax
from connecting_db import connection
from query_functions import *
from collections import Counter

In [10]:
def add_padding(str1: str, str2: str):

    """
    This function adds zero padding to the left in the smallest string
    """

    max_size = max(len(str1), len(str2))
    return str1.zfill(max_size), str2.zfill(max_size)

def truncate_strings(str1: str, str2: str, right_truncation: bool = True): 

    """
    This function truncates the longer string to meet the smaller string's size. 
    It can be truncated to the left or to the right.
    """
    min_size = min(len(str1), len(str2))  

    if right_truncation:

        return str1[:min_size], str2[:min_size]
    
    else: 
        return str1[-min_size:], str2[-min_size:]

        
def get_id(str1: str, case_sensitive: bool = True, *args, **kwargs):

    """
    This function converts the string to a numpy array composed of the ids from the characters
    
    """
    if not case_sensitive:
        str1 = str1.upper()
    
    str1_array = np.array([ord(i) for i in str1])

    return str1_array
    


In [11]:
def compare_strings(str1: str, str2: str, padding: bool = True, case_sensitive: bool = True, *args, **kwargs):

    """
    This function compares both strings. Each character of both strings is compared with the character in the same position of the other string.

    If the strings have different sizes:

    padding = True -> We pad the smaller string with zeros to the left
    padding = False -> We truncate the string to the left or to the right (right_truncation = True or False)

    We can compare with case sensitive or not

    case_sensitive = True or False 

    """

    if padding:
        str1, str2 = add_padding(str1, str2, *args, **kwargs)
    else:
        str1, str2 = truncate_strings(str1, str2, *args, **kwargs)
   
    str1_id = get_id(str1, case_sensitive, *args, **kwargs)
    str2_id = get_id(str2, case_sensitive, *args, **kwargs)

    return np.mean(str1_id == str2_id)

def is_contained(str1: str, str2: str, case_sensitive: bool = True):
    """
    This function checks if the smaller string is contained in the bigger string
    
    """
    if not case_sensitive:
        str1 = str1.upper()
        str2 = str2.upper()

    if str1 < str2:
        return str1 in str2
    else: 
        return str2 in str1

def is_anagram(str1: str, str2: str, case_sensitive: bool = True, right_truncation: bool = True):

    """
    This function checks if the strings are anagrams of each other.

    The test can be case sensitive or not

    If the strings have different sizes, it is possible to truncate the bigger one on the right or on the left side
    """

    if not case_sensitive:
        str1 = str1.upper()
        str2 = str2.upper()

    str1, str2 = truncate_strings(str1, str2, right_truncation=right_truncation)



    str1_dict = Counter(str1)
    str2_dict = Counter(str2)

    return str1_dict == str2_dict

def is_anagram_combined(str1: str, str2: str, case_sensitive: bool = True):

    """
    This function combines the test truncating the bigger string in the right and the left.

    The result is an OR of both results since we want to know if it is an anagram in any way.
    """

    right_truncated = is_anagram(str1, str2, case_sensitive, right_truncation=True )
    left_truncated = is_anagram(str1, str2, case_sensitive, right_truncation=False )

    return right_truncated or left_truncated


def log_difference(str1: str, str2: str,  padding: bool = True, case_sensitive: bool = True, *args, **kwargs):

    """
    
    This function calculates the euclidian distance of the words using the ASCII table as a reference.

    For example, the letter a from letter b are 1 unit apart. That means, the word 'ab' is sqrt((2-1)^2 + (3-2)^2) units apart from the word 'bc'
    
    """

    if padding:

        size_difference = abs(len(str1) - len(str2))
        concat_array = np.zeros(size_difference)

        str1_id = get_id(str1, case_sensitive, *args, **kwargs)
        str2_id = get_id(str2, case_sensitive, *args, **kwargs)
 

        if len(str1_id)>len(str2_id):
            str2_id = np.concatenate((str2_id, concat_array))

        elif len(str1_id)<len(str2_id):
            str1_id = np.concatenate((str1_id, concat_array))


    else:
        str1, str2 = truncate_strings(str1, str2, *args, **kwargs)
   
        str1_id = get_id(str1, case_sensitive, *args, **kwargs)
        str2_id = get_id(str2, case_sensitive, *args, **kwargs)
        
    difference = np.linalg.norm(str1_id- str2_id)
  

    if difference != 0:
        return np.log(difference)
    else:
        return difference

In [12]:
def log_difference_combined(str1: str, str2: str, case_sensitive: bool = True):

    """
    This function only combines all possible ways to measure the log distance, and returns a weighted average
    """

    padding_pontuation = log_difference(str1, str2, padding=True,
     case_sensitive=case_sensitive)
    left_pontuation = log_difference(str1, str2, padding=False, case_sensitive=case_sensitive,right_truncation=False)
    right_pontuation = log_difference(str1, str2, padding=False,case_sensitive=case_sensitive, right_truncation=True)
    points = np.array([.5*padding_pontuation, .25*left_pontuation, .25*right_pontuation])
    return np.linalg.norm(points)

def compare_strings_combined(str1: str, str2: str, case_sensitive: bool = True):

    """

    This function combines all ways to compare strings
    
    """

    similiarity_padding = compare_strings(str1, str2, padding= True, case_sensitive=case_sensitive)
    similiarity_right = compare_strings(str1, str2, padding= False, case_sensitive=case_sensitive, right_truncation=True)
    similiarity_left = compare_strings(str1, str2, padding= False, case_sensitive=case_sensitive, right_truncation=False)
    is_contained_ = is_contained(str1, str2, case_sensitive=case_sensitive)

    # print([similiarity_padding,similiarity_right,similiarity_left, is_contained_])
    return np.mean([similiarity_padding,similiarity_right,similiarity_left, is_contained_] )

In [13]:
def comparsion_metrics(str1: str, str2: str, case_sensitive: bool = True):


    """

    This function returns all metrics in one list. We input 2 strings and get all possible metrics for these two strings.

    """
    return [compare_strings_combined(str1, str2, case_sensitive), log_difference_combined(str1, str2, case_sensitive), is_anagram_combined(str1, str2)*1]
    # return [compare_strings_combined(str1, str2, case_sensitive), log_difference(str1, str2, case_sensitive), is_anagram_combined(str1, str2)*1]

def search_similar(str1: str, search_list: list, case_sensitive: bool = True):


    """
    This function runs through a list of strings to compare with the targeted strings and returns the most similar string in the list for the metrics used.

    It is useful to search for a similar string in a database of strings.
    
    """

    similarities = np.array([comparsion_metrics(str1, str2, case_sensitive) for str2 in search_list])
    most_similar_points = round(similarities[:, 0].max(),4)
    most_similar_coordinates = similarities[:, 0].argmax()
    closest_points = round(similarities[:, 1].min(),4)
    closest_coordinates = similarities[:, 1].argmin()

    anagram_max = similarities[:,2].max()
    anagram_max_coordinates = similarities[:,2].argmax()

    if anagram_max == 0:
        anagram_serial = None
    else:
        anagram_serial = search_list[anagram_max_coordinates]
        

    return [most_similar_points, search_list[most_similar_coordinates], closest_points, search_list[closest_coordinates], anagram_serial]

## Randomize String Functions

Functions to randomize strings. Used to cover confidential information during tests.

In [ ]:
import secrets
sysrand = secrets.SystemRandom()

def shuffle_string(str_: str, add_characters: bool = False, add_characters_byte_size: int = 1):

    """

    This function shuffles the strings into a new string. It is also possible to add characters to make the strings even more different than the original ones

    """

    shuffle_string = str_

    if add_characters:
        shuffle_string =shuffle_string + secrets.token_hex(add_characters_byte_size).upper()
    
    shuffle_string = list(shuffle_string)
    sysrand.shuffle(shuffle_string)

    shuffle_string = ''.join(shuffle_string)
    return shuffle_string

def random_shuffle(str_: str):

    """
 This function randomizes strings in three different random ways:

        1. Shuffles the original string
        2. Shuffles and add characters to the original string
        3. Adds new characters to the original string
    """
    random_flag = sysrand.randint(1,3)
    
    if random_flag == 1:
        str_ = shuffle_string(str_)
    elif random_flag == 2:
        str_ = shuffle_string(str_, add_characters= True)
    else:
        str_ = str_ + secrets.token_hex(1).upper()
    
    return str_

## Comparing strings

### Reading the base data

In [4]:
nlyte_query = read_query_file('./query_files/query_maximo_nlyte.sql')

nlyte_assets = pd.read_sql_query(nlyte_query, connection)

### Removing blanks and void serials

In [15]:
nlyte_assets.SerialNumber.fillna('nan', inplace=True)
nlyte_assets.SerialNumber.replace({ '': 'nan'}, inplace=True)

### Testing the algorithm in a small sample in the same database using anonymized data

In [108]:
sample_size = 250
a = []
search_list = nlyte_assets.SerialNumber.sample(sample_size).tolist()

search_list = [random_shuffle(i) for i in search_list] ## Making new random serial numbers to hide original ones
new_sample = [random_shuffle(i) for i in search_list] ## Derivating a test list from the original
# new_sample = nlyte_assets.SerialNumber.sample(sample_size).tolist()
for i in new_sample:
    a.append([i] + search_similar(i, search_list, False))
pd.DataFrame(a, columns=['original_serial',
                         'similarity_percentage', 
                         'most_similar', 
                         'log_distance', 
                         'closeest', 'anagram'])

,original_serial,similarity_percentage,most_similar,log_distance,closeest,anagram
0,B3112DP1964004E5,0.1897,BW074T5,2.2559,60H022T411310209,None
1,25CCQNM,0.2143,2541457,1.8139,728GPCD,NCQ5C2M
2,833NEAA0TC6YC25,0.1738,NCQ5C2M,2.5288,ER1T21BRH.2MO00,None
3,1Q618KHBR,0.1905,61ZNHP2,2.1782,7821HCNBD,R8B1H1KQ6
4,A2517A445,0.1667,50C17651P,1.6781,782478325,None
...,...,...,...,...,...,...
245,50E879278,0.2500,50C17651P,1.5539,50B79826D,None
246,92QWGVTKPEB,0.5000,92QWGVTKP,2.3976,61RF2LJ1C01,92QWGVTKP
247,7701002101A407,0.5774,7701002101A4,2.1166,18000207F0052C,7701002101A4
248,0078082Z2FAF,0.6792,0078082Z2F,2.1132,80B547CN19GR,0078082Z2F


### Reading the database where the similarities are going to be searched

In [8]:
csv =  False

file_name = 'MAXIMO_SHARED'


columns_to_keep = ['ASSET_NUMBER',
                    'CUSTOMER',
                    'ASSET_STATUS',
                    'MODEL',
                    'REFERENCE_SERIAL_NUMBER',
                    'CITY',
                    'ROOM',
                    'RACK',
                    'KVA',
                    'HW_ELIGIBLE_BY_NLYTE',
                    'HW_NLYTE_HISTORY',
                    'HW_NLYTE_LASTSCAN',
                    'HW_NLYTE_UPDATE',
                    'HW_MATERIAL_NAME',]
if csv:
    maximo = pd.read_excel(f'./{file_name}.xlsx', sheet_name="Sheet1")
    maximo = maximo.loc[:, columns_to_keep]
    maximo.to_pickle(f'{file_name}.pkl.gz')
else:
    maximo = pd.read_pickle(f'{file_name}.pkl.gz')

maximo = maximo.loc[:, columns_to_keep]
maximo.REFERENCE_SERIAL_NUMBER = maximo.REFERENCE_SERIAL_NUMBER.apply(str)

###  Removing blanks and void serials

In [20]:
maximo.REFERENCE_SERIAL_NUMBER.fillna('nan', inplace=True)
maximo.REFERENCE_SERIAL_NUMBER.replace({'': 'nan'}, inplace=True)

### Filtering only the clients where there was a scan

In [18]:
clients = set(maximo[maximo.HW_NLYTE_LASTSCAN.notna() & maximo.HW_NLYTE_LASTSCAN.notnull()].CUSTOMER.unique())

In [21]:
nlyte_inventory = maximo.query('ASSET_STATUS == "NLYTE INVENTORY" and CUSTOMER.isin(@clients)').REFERENCE_SERIAL_NUMBER.to_list()

In [22]:
not_updated = maximo.query('HW_ELIGIBLE_BY_NLYTE == "Y" and HW_NLYTE_UPDATE.isna() and CUSTOMER.isin(@clients) and REFERENCE_SERIAL_NUMBER != "C"').REFERENCE_SERIAL_NUMBER.to_list()

### Searching the database for similarities and saving into a data frame

In [ ]:
assets = []
for asset in nlyte_inventory:
    assets.append([asset] + search_similar(asset, not_updated, False))

df = pd.DataFrame(assets, columns=['original_serial',
                         'similarity_percentage', 
                         'most_similar', 
                         'log_distance', 
                         'closeest', 'anagram'])

In [24]:
pd.DataFrame(assets, columns=['original_serial',
                         'similarity_percentage', 
                         'most_similar', 
                         'log_distance', 
                         'closeest', 'anagram']).to_excel('comparsion.xlsx', index=False)